In [ ]:
"""
Industrial-grade stock price prediction system.

This module provides a robust framework for downloading stock data,
training prediction models, and generating forecasts with proper
error handling and logging.
"""

import logging
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score


@dataclass
class PredictionConfig:
    """Configuration for stock prediction parameters."""
    
    stocks: List[str]
    training_years: int = 5
    forecast_days: int = 10
    validation_splits: int = 5
    plot_months: int = 4
    output_dir: str = "results"
    log_level: str = "INFO"


@dataclass
class ModelMetrics:
    """Container for model performance metrics."""
    
    accuracy: float
    r2_score: float
    mape: float
    cv_score_mean: float
    cv_score_std: float


class StockDataManager:
    """Handles stock data downloading and preprocessing."""
    
    def __init__(self, logger: logging.Logger):
        self.logger = logger
    
    def download_stock_data(
        self, 
        ticker: str, 
        start_date: datetime, 
        end_date: datetime
    ) -> Optional[pd.DataFrame]:
        """
        Download and preprocess stock data for a given ticker.
        
        Args:
            ticker: Stock ticker symbol
            start_date: Start date for data retrieval
            end_date: End date for data retrieval
            
        Returns:
            DataFrame with processed stock data or None if failed
        """
        try:
            self.logger.info(f"Downloading data for {ticker}")
            
            # Download data with retry logic
            df = yf.download(ticker, start=start_date, end=end_date, progress=False)
            
            if df.empty:
                self.logger.warning(f"No data retrieved for {ticker}")
                return None
            
            # Select and clean data
            df = df[['Close']].copy()
            df.dropna(inplace=True)
            
            if len(df) < 100:  # Minimum data requirement
                self.logger.warning(f"Insufficient data for {ticker}: {len(df)} records")
                return None
            
            self.logger.info(f"Successfully downloaded {len(df)} records for {ticker}")
            return df
            
        except Exception as e:
            self.logger.error(f"Failed to download data for {ticker}: {str(e)}")
            return None


class StockPredictor:
    """Handles model training and prediction for stock prices."""
    
    def __init__(self, logger: logging.Logger):
        self.logger = logger
        self.model = LinearRegression()
    
    def prepare_features(
        self, 
        df: pd.DataFrame, 
        forecast_days: int
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare features and targets for model training.
        
        Args:
            df: DataFrame with stock data
            forecast_days: Number of days to forecast
            
        Returns:
            Tuple of (features, targets)
        """
        # Create shifted target variable
        df_copy = df.copy()
        df_copy['Target'] = df_copy['Close'].shift(-forecast_days)
        
        # Remove rows with NaN targets
        valid_data = df_copy.dropna()
        
        X = valid_data[['Close']].values
        y = valid_data['Target'].values
        
        return X, y
    
    def train_model(self, X: np.ndarray, y: np.ndarray) -> ModelMetrics:
        """
        Train the prediction model and calculate performance metrics.
    
        Args:
            X: Feature matrix
            y: Target vector
        
        Returns:
            ModelMetrics object with performance statistics
        """
        try:
            # Split data into train (80%) and test (20%) sets
            # For time series data, we use the first 80% for training
            split_idx = int(0.8 * len(X))
            X_train, X_test = X[:split_idx], X[split_idx:]
            y_train, y_test = y[:split_idx], y[split_idx:]
    
            # Train the model on training data
            self.model.fit(X_train, y_train)
    
            # Make predictions on test data (unseen data)
            y_pred = self.model.predict(X_test)
        
            # Calculate metrics on test set (not full dataset)
            mape = mean_absolute_percentage_error(y_test, y_pred) * 100
            accuracy = max(0, 100 - mape)  # Ensure non-negative accuracy
            r2 = r2_score(y_test, y_pred)
        
            # Cross-validation on training data for more robust evaluation
            tscv = TimeSeriesSplit(n_splits=5)
            cv_scores = cross_val_score(
                self.model, X_train, y_train, cv=tscv, scoring='neg_mean_absolute_percentage_error')
            cv_score_mean = -cv_scores.mean() * 100
            cv_score_std = cv_scores.std() * 100
        
            return ModelMetrics(
                accuracy=accuracy,
                r2_score=r2,
                mape=mape,
                cv_score_mean=cv_score_mean,
                cv_score_std=cv_score_std)
        except Exception as e:
            self.logger.error(f"Model training failed: {str(e)}")
            raise
    
    def generate_forecast(
        self, 
        df: pd.DataFrame, 
        forecast_days: int
    ) -> pd.DataFrame:
        """
        Generate price forecasts for the specified number of days.
        
        Args:
            df: DataFrame with stock data
            forecast_days: Number of days to forecast
            
        Returns:
            DataFrame with forecast results
        """
        try:
            # Get the most recent prices for prediction
            recent_prices = df['Close'].tail(forecast_days).values.reshape(-1, 1)
            
            # Generate predictions
            forecast = self.model.predict(recent_prices)
            
            # Create forecast DataFrame
            forecast_dates = pd.date_range(
                start=df.index[-forecast_days],
                periods=forecast_days,
                freq='D'
            )
            
            forecast_df = pd.DataFrame({
                'Predicted_Price': forecast
            }, index=forecast_dates)
            
            return forecast_df
            
        except Exception as e:
            self.logger.error(f"Forecast generation failed: {str(e)}")
            raise


class ResultsVisualizer:
    """Handles visualization and result presentation."""
    
    def __init__(self, logger: logging.Logger, output_dir: str):
        self.logger = logger
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
    
    def plot_predictions(
        self,
        ticker: str,
        df: pd.DataFrame,
        forecast_df: pd.DataFrame,
        metrics: ModelMetrics,
        plot_months: int
    ) -> None:
        """
        Create and save prediction visualization.
        
        Args:
            ticker: Stock ticker symbol
            df: Historical stock data
            forecast_df: Forecast results
            metrics: Model performance metrics
            plot_months: Number of months to show in plot
        """
        try:
            plt.figure(figsize=(14, 8))
            
            # Filter recent data for plotting
            cutoff_date = df.index[-1] - timedelta(days=plot_months * 30)
            recent_data = df[df.index >= cutoff_date]
            
            # Plot historical and predicted prices
            plt.plot(
                recent_data.index, 
                recent_data['Close'], 
                label='Historical Price', 
                linewidth=2,
                color='blue'
            )
            
            plt.plot(
                forecast_df.index, 
                forecast_df['Predicted_Price'], 
                label='Predicted Price', 
                linestyle='--', 
                linewidth=2, 
                marker='o',
                color='red',
                markersize=6
            )
            
            # Formatting
            plt.title(
                f'{ticker} Stock Price Prediction\n'
                f'Accuracy: {metrics.accuracy:.1f}% | R²: {metrics.r2_score:.3f} | '
                f'CV MAPE: {metrics.cv_score_mean:.1f}±{metrics.cv_score_std:.1f}%',
                fontsize=14,
                fontweight='bold'
            )
            
            plt.xlabel('Date', fontsize=12)
            plt.ylabel('Price ($)', fontsize=12)
            plt.legend(fontsize=11)
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            # Save plot
            plot_path = self.output_dir / f"{ticker}_prediction.png"
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.show()
            
            self.logger.info(f"Plot saved to {plot_path}")
            
        except Exception as e:
            self.logger.error(f"Plotting failed for {ticker}: {str(e)}")
    
    def save_results_summary(
        self, 
        results: Dict[str, Tuple[pd.DataFrame, ModelMetrics]]
    ) -> None:
        """
        Save a summary of all prediction results to CSV.
        
        Args:
            results: Dictionary mapping tickers to (forecast_df, metrics) tuples
        """
        try:
            summary_data = []
            
            for ticker, (forecast_df, metrics) in results.items():
                summary_data.append({
                    'Ticker': ticker,
                    'Accuracy (%)': round(metrics.accuracy, 2),
                    'R² Score': round(metrics.r2_score, 4),
                    'MAPE (%)': round(metrics.mape, 2),
                    'CV MAPE Mean (%)': round(metrics.cv_score_mean, 2),
                    'CV MAPE Std (%)': round(metrics.cv_score_std, 2),
                    'Latest Prediction ($)': round(forecast_df['Predicted_Price'].iloc[-1], 2)
                })
            
            summary_df = pd.DataFrame(summary_data)
            summary_path = self.output_dir / "prediction_summary.csv"
            summary_df.to_csv(summary_path, index=False)
            
            self.logger.info(f"Results summary saved to {summary_path}")
            print("\n" + "="*60)
            print("PREDICTION SUMMARY")
            print("="*60)
            print(summary_df.to_string(index=False))
            
        except Exception as e:
            self.logger.error(f"Failed to save results summary: {str(e)}")


class StockPredictionSystem:
    """Main system class orchestrating the entire prediction pipeline."""
    
    def __init__(self, config: PredictionConfig):
        self.config = config
        
        # Create output directory first
        self.output_path = Path(config.output_dir)
        self.output_path.mkdir(parents=True, exist_ok=True)
        
        # Setup logging after directory creation
        self.logger = self._setup_logging()
        
        # Initialize components
        self.data_manager = StockDataManager(self.logger)
        self.predictor = StockPredictor(self.logger)
        self.visualizer = ResultsVisualizer(self.logger, config.output_dir)
    
    def _setup_logging(self) -> logging.Logger:
        """Set up logging configuration."""
        # Get logger for this module
        logger = logging.getLogger(__name__)
        
        # Clear existing handlers to avoid duplicates
        if logger.handlers:
            logger.handlers.clear()
        
        # Set level
        logger.setLevel(getattr(logging, self.config.log_level))
        
        # Create formatter
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        
        # Always add console handler
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
        
        # Try to add file handler
        try:
            log_file = self.output_path / "prediction.log"
            file_handler = logging.FileHandler(log_file, mode='w')  # 'w' to overwrite
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)
            print(f"📝 Logging to: {log_file}")
        except Exception as e:
            print(f"⚠️  Could not create log file: {e}")
            print("📺 Using console logging only")
        
        # Prevent propagation to root logger
        logger.propagate = False
        
        return logger
    
    def run_predictions(self) -> Dict[str, Tuple[pd.DataFrame, ModelMetrics]]:
        """
        Execute the complete prediction pipeline for all configured stocks.
        
        Returns:
            Dictionary mapping tickers to (forecast_df, metrics) tuples
        """
        self.logger.info("Starting stock prediction system")
        
        # Calculate date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * self.config.training_years)
        
        results = {}
        successful_predictions = 0
        
        for ticker in self.config.stocks:
            try:
                self._process_single_stock(ticker, start_date, end_date, results)
                successful_predictions += 1
                
            except Exception as e:
                self.logger.error(f"Failed to process {ticker}: {str(e)}")
                continue
        
        self.logger.info(
            f"Completed predictions for {successful_predictions}/{len(self.config.stocks)} stocks"
        )
        
        # Save summary if we have results
        if results:
            self.visualizer.save_results_summary(results)
        
        return results
    
    def _process_single_stock(
        self,
        ticker: str,
        start_date: datetime,
        end_date: datetime,
        results: Dict
    ) -> None:
        """Process a single stock through the complete pipeline."""
        print(f"\n📊 Processing {ticker}...")
        
        # Download data
        df = self.data_manager.download_stock_data(ticker, start_date, end_date)
        if df is None:
            raise ValueError(f"Failed to download data for {ticker}")
        
        # Prepare features
        X, y = self.predictor.prepare_features(df, self.config.forecast_days)
        if len(X) == 0:
            raise ValueError(f"Insufficient data after preprocessing for {ticker}")
        
        # Train model and get metrics
        metrics = self.predictor.train_model(X, y)
        
        # Generate forecast
        forecast_df = self.predictor.generate_forecast(df, self.config.forecast_days)
        
        # Display results
        self._display_results(ticker, forecast_df, metrics)
        
        # Create visualization
        self.visualizer.plot_predictions(
            ticker, df, forecast_df, metrics, self.config.plot_months
        )
        
        # Store results
        results[ticker] = (forecast_df, metrics)
    
    def _display_results(
        self, 
        ticker: str, 
        forecast_df: pd.DataFrame, 
        metrics: ModelMetrics
    ) -> None:
        """Display prediction results in a formatted manner."""
        print(f"\n📈 {ticker} - Predicted Prices for next {self.config.forecast_days} days:")
        print("-" * 50)
        for date, price in forecast_df['Predicted_Price'].items():
            print(f"{date.strftime('%Y-%m-%d')}: ${price:.2f}")
        
        print(f"\n🎯 Model Performance:")
        print(f"   Training Accuracy: {metrics.accuracy:.2f}%")
        print(f"   R² Score: {metrics.r2_score:.4f}")
        print(f"   Training MAPE: {metrics.mape:.2f}%")
        print(f"   Cross-Validation MAPE: {metrics.cv_score_mean:.2f}±{metrics.cv_score_std:.2f}%")


def main():
    """Main entry point for the stock prediction system."""
    
    # Configuration
    config = PredictionConfig(
        stocks=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'],
        training_years=5,
        forecast_days=10,
        validation_splits=5,
        plot_months=4,
        output_dir="stock_predictions",
        log_level="INFO"
    )
    
    # Run predictions
    system = StockPredictionSystem(config)
    results = system.run_predictions()
    
    print(f"\n✅ Prediction pipeline completed successfully!")
    print(f"📁 Results saved to: {config.output_dir}/")


if __name__ == "__main__":
    main()

2025-05-28 11:15:18,692 - __main__ - INFO - Starting stock prediction system
2025-05-28 11:15:18,695 - __main__ - INFO - Downloading data for AAPL


📝 Logging to: stock_predictions\prediction.log

📊 Processing AAPL...


2025-05-28 11:15:19,234 - __main__ - INFO - Successfully downloaded 1255 records for AAPL
2025-05-28 11:15:19,245 - __main__ - ERROR - Model training failed: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:19,247 - __main__ - ERROR - Failed to process AAPL: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:19,249 - __main__ - INFO - Downloading data for MSFT



📊 Processing MSFT...


2025-05-28 11:15:19,918 - __main__ - INFO - Successfully downloaded 1255 records for MSFT
2025-05-28 11:15:19,928 - __main__ - ERROR - Model training failed: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:19,930 - __main__ - ERROR - Failed to process MSFT: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:19,931 - __main__ - INFO - Downloading data for GOOGL


...


📊 Processing AMZN...


2025-05-28 11:15:20,684 - __main__ - INFO - Successfully downloaded 1255 records for AMZN
2025-05-28 11:15:20,697 - __main__ - ERROR - Model training failed: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:20,699 - __main__ - ERROR - Failed to process AMZN: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:20,702 - __main__ - INFO - Downloading data for TSLA



📊 Processing TSLA...


2025-05-28 11:15:21,095 - __main__ - INFO - Successfully downloaded 1255 records for TSLA
2025-05-28 11:15:21,107 - __main__ - ERROR - Model training failed: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:21,109 - __main__ - ERROR - Failed to process TSLA: Found input variables with inconsistent numbers of samples: [1245, 249]
2025-05-28 11:15:21,111 - __main__ - INFO - Completed predictions for 0/5 stocks



✅ Prediction pipeline completed successfully!
📁 Results saved to: stock_predictions/


Something something etc 